In [12]:
import pandas
import numpy
import os
import math
import urllib
from datetime import datetime


BOLD = '\033[1m'
END = '\033[0m'

#Marchenko rodion Data Analisys prakt №2

print(BOLD+"MARCHENKO RODION FB-23 PRAKT №2 \"Dataset downloading and basic aggregate functions\"")
print("="*83+END)


#This function loads dataset CSV table from the internet
def LoadClimateDatasetAsCSV(WorkDir,RegionID,BeginYear,EndYear):
    url = "https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID="+str(RegionID)+"&country=UKR&yearlyTag=Weekly&type=Mean&TagCropland=crop&year1="+str(BeginYear)+"&year2="+str(EndYear)
    CurrentTime = datetime.now().strftime("%d-%m-%y-%H-%M-%S")
    Filename = WorkDir+"/Vegetation-health-index-UKR-provinceNo"+str(RegionID)+"--"+CurrentTime+".csv"
    urllib.request.urlretrieve(url,Filename)
    print("\t» Downloaded: Vegetation-health-index-UKR-provinceNo"+str(RegionID)+"--"+CurrentTime+".csv")
    return Filename



#"MAIN" FUNCTION:
#Create temporary directory if it does not exist
CurrentDirectory = os.getcwd()
if (os.getcwd()[-13:] != "/DataLab2-tmp"):
    try:
        os.chdir(CurrentDirectory+"/DataLab2-tmp")
    except:
        try:
            os.makedirs(CurrentDirectory+"/DataLab2-tmp")
            print("»» Created tmp directory /DataLab2-tmp.")
        except FileExistsError:
            # directory already exists
            pass
        os.chdir(CurrentDirectory+"/DataLab2-tmp")
 
print("»» The current working directory is: "+os.getcwd()+".")
print("\n")


#Loading vegetation data for all provinces of Ukraine
CWD = os.getcwd()
AvailableFiles = os.listdir(CWD)
for province in range(1,28):
    if any("Vegetation-health-index-UKR-provinceNo"+str(province) in path for path in AvailableFiles):
        pass
    else:
        try:
            Filename = LoadClimateDatasetAsCSV(CWD,province,1982,2024)
        except:
            print("\t» Failed to load for province №",province)
print("»» Loaded all!!!")


MARCHENKO RODION FB-23 PRAKT №2 "Dataset downloading and basic aggregate functions"
»» The current working directory is: /home/rodion/DataLab2-tmp.


»» Loaded all!!!


In [13]:

#This function reads datasets as pandas frames and performs some preprocessing
def ReadDatasetToCSV(WorkDir):
    DataFrames = []
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty'] #CSV table headers
    print("\n»» "+BOLD+"Loading data from files:"+END)
    for path in os.listdir(WorkDir):
        FullPath = os.path.join(WorkDir, path)
        if (os.path.isfile(FullPath) and path[-4:] == ".csv"): # check if current path is a CSV file
            print(path)
            df = pandas.read_csv(FullPath, header = 1, names = headers)
            df = df.drop(df.loc[df["VHI"] == -1].index) #Drop empty value rows for VHI
            df = df.drop(labels="empty", axis=1) #Drop empty column
            df.loc[0, "Year"] = df["Year"][1] #Fix year value in first row (get rid of html tags)
            df = df.dropna(subset=["VHI"]) #Drop rows with NaN in for VHI
            df = df.astype({"Year": "int32"}) #Convert year column to integer for easy indexation

            pathNameArray = path.split("-")
            df["province"] = int(pathNameArray[4][10:]) #Extract province code from predetermined file name and add it as a column
            
            DataFrames.append(df)
    FullData = pandas.concat(DataFrames) #Coalesce all data into a single dataframe
    FullData = FullData.reset_index(drop=True) #Reset global index without preserving the old one
    print("\n"+"="*75)
    print(FullData)
    return FullData


FullDataSet = ReadDatasetToCSV(os.getcwd())



»» Loading data from files:
Vegetation-health-index-UKR-provinceNo8--16-03-24-09-14-40.csv
Vegetation-health-index-UKR-provinceNo16--16-03-24-09-14-48.csv
Vegetation-health-index-UKR-provinceNo13--16-03-24-09-14-45.csv
Vegetation-health-index-UKR-provinceNo15--16-03-24-09-14-47.csv
Vegetation-health-index-UKR-provinceNo18--16-03-24-09-14-50.csv
Vegetation-health-index-UKR-provinceNo2--16-03-24-09-14-35.csv
Vegetation-health-index-UKR-provinceNo24--16-03-24-09-14-55.csv
Vegetation-health-index-UKR-provinceNo1--16-03-24-09-14-34.csv
Vegetation-health-index-UKR-provinceNo19--16-03-24-09-14-50.csv
Vegetation-health-index-UKR-provinceNo7--16-03-24-09-14-39.csv
Vegetation-health-index-UKR-provinceNo10--16-03-24-09-17-24.csv
Vegetation-health-index-UKR-provinceNo14--16-03-24-09-14-46.csv
Vegetation-health-index-UKR-provinceNo4--16-03-24-09-14-37.csv
Vegetation-health-index-UKR-provinceNo22--16-03-24-09-14-53.csv
Vegetation-health-index-UKR-provinceNo6--16-03-24-09-14-38.csv
Vegetation-health

In [14]:

#This function replaces province IDs inside dataframe for a more readable Name Identifier
def ProvinceIdToNames(FullData):
    ReplacementsDict = {1:22, 2:24, 3:23, 4:25, 5:3, 6:4, 7:8, 8:19, 9:20, 10:21, 11:9, 12:26, 13:10, 14:11, 15:12, 16:13, 17:14, 18:15, 19:16, 20:27, 21:17, 22:18, 23:6, 24:1, 25:2, 26:7, 27:5}
    FullData["province"].replace(ReplacementsDict, inplace = True)
    
    ReplacementsDict2 = {1:"Вінницька", 2:"Волинська", 3:"Дніпропетровська", 4:"Донецька", 5:"Житомирська", 6:"Закарпатська", 7:"Запорізька", 8:"Івано-Франківська", 9:"Київська", 10:"Кіровоградська", 11:"Луганська", 12:"Львівська", 13:"Миколаївська", 14:"Одеська", 15:"Полтавська", 16:"Рівенська", 17:"Сумська", 18:"Тернопільська", 19:"Харківська", 20:"Херсонська", 21:"Хмельницька", 22:"Черкаська", 23:"Чернівецька", 24:"Чернігівська", 25:"Республіка Крим", 26:"м. Київ", 27:"м. Севастополь"}
    FullData["province"].replace(ReplacementsDict2, inplace = True)
    print("\n"+"="*75)
    print(FullData)


ProvinceIdToNames(FullDataSet)



       Year  Week    SMN     SMT    VCI    TCI    VHI     province
0      1982   1.0  0.038  252.16  46.81  69.25  58.03   Харківська
1      1982   2.0  0.035  253.31  45.08  63.66  54.37   Харківська
2      1982   3.0  0.032  254.12  40.84  60.41  50.63   Харківська
3      1982   4.0  0.030  255.48  36.81  58.22  47.52   Харківська
4      1982   5.0  0.029  257.04  32.00  57.13  44.57   Харківська
...     ...   ...    ...     ...    ...    ...    ...          ...
57883  2024   6.0  0.097  265.90  57.84  36.80  47.32  Житомирська
57884  2024   7.0  0.114  268.92  63.05  30.65  46.85  Житомирська
57885  2024   8.0  0.134  271.78  69.82  25.86  47.84  Житомирська
57886  2024   9.0  0.153  274.49  74.69  22.35  48.52  Житомирська
57887  2024  10.0  0.167  276.05  74.97  24.54  49.76  Житомирська

[57888 rows x 8 columns]


/tmp/ipykernel_49628/716397667.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  FullData["province"].replace(ReplacementsDict, inplace = True)
/tmp/ipykernel_49628/716397667.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplac

In [15]:

#This function finds anual extremes of VHI index for a given province and year
def AnualExtremeVHI(FullData,ProvinceName,Year):
    yearset = FullData[(FullData["province"] == ProvinceName) & (FullData["Year"] == Year)]["VHI"]
    MinValue = yearset.min()
    MinValueIndex = yearset[yearset == MinValue].index[0]
    MinValueWeek = int(FullData.iloc[MinValueIndex]["Week"])
    MaxValue = yearset.max()
    MaxValueIndex = yearset[yearset == MaxValue].index[0]
    MaxValueWeek = int(FullData.iloc[MaxValueIndex]["Week"])
    
    print("\n")
    print(yearset.head(10))
    print("="*83)
    print(BOLD+"\tThe minimal VHI index =",MinValue,"on week",MinValueWeek,END+" at index",MinValueIndex,"in dataset.")
    print(BOLD+"\tThe maximal VHI index =",MaxValue,"on week",MaxValueWeek,END+" at index",MaxValueIndex,"in dataset.")
    return [MinValue,MaxValue,MinValueWeek,MaxValueWeek,MinValueIndex,MaxValueIndex]


extreme1 = AnualExtremeVHI(FullDataSet,"Кіровоградська",2023)
extreme2 = AnualExtremeVHI(FullDataSet,"Луганська",2023)




6370    43.73
6371    45.25
6372    46.11
6373    46.37
6374    45.05
6375    44.98
6376    45.36
6377    45.48
6378    45.27
6379    43.45
Name: VHI, dtype: float64
	The minimal VHI index = 24.17 on week 43  at index 6412 in dataset.
	The maximal VHI index = 69.4 on week 29  at index 6398 in dataset.


25666    56.89
25667    56.77
25668    54.21
25669    52.47
25670    47.91
25671    43.27
25672    40.97
25673    41.56
25674    42.09
25675    43.05
Name: VHI, dtype: float64
	The minimal VHI index = 31.31 on week 44  at index 25709 in dataset.
	The maximal VHI index = 74.64 on week 26  at index 25691 in dataset.


In [16]:

#This function finds extreme or mild years of low VHI index in all dataset for a given percentage of affected territory
def AlltimeVHIbyProvincePercent(FullData, BeginYear, EndYear, AreaPercent, Severity):
    if Severity == "severe":
        MAXVHI = 15
    elif Severity == "mild":
        MAXVHI = 35
    DroughtsSet = FullData[(FullData["VHI"] <= MAXVHI) & (FullData["VHI"] >= 0) & (FullData["Year"] >= BeginYear) & (FullData["Year"] <= EndYear)]
    DroughtsByYear = DroughtsSet.groupby(["Year"])
    DroughtProvincesByYear = DroughtsByYear["province"].nunique() #Get number of provinces affected by drought
    NumberOfProvinces = round((AreaPercent*len(FullData["province"].unique()))/100) #Calculate the province count based on total area percent
    
    print("\n")
    print("The annual number of provinces affected by drought is:")
    print(DroughtProvincesByYear)
    print("="*83)
    print("The "+Severity+"ly dry years for "+BOLD+str(AreaPercent)+"%"+END+" of land area are:")
    years = list(DroughtProvincesByYear[DroughtProvincesByYear >= NumberOfProvinces].index) #Get only years with more than certain amount of provinces
    print(BOLD,years,END)
    return years


years1 = AlltimeVHIbyProvincePercent(FullDataSet,1992,2008,20,"severe")
years2 = AlltimeVHIbyProvincePercent(FullDataSet,1992,2008,75,"mild")




The annual number of provinces affected by drought is:
Year
1992    2
1993    3
1994    3
1999    1
2000    6
2003    2
2007    5
Name: province, dtype: int64
The severely dry years for 20% of land area are:
 [2000, 2007] 


The annual number of provinces affected by drought is:
Year
1992    25
1993    25
1994    23
1995    26
1996    27
1997    18
1998     8
1999    22
2000    27
2001    15
2002    12
2003    16
2004     3
2005     8
2006    14
2007    18
2008    18
Name: province, dtype: int64
The mildly dry years for 75% of land area are:
 [1992, 1993, 1994, 1995, 1996, 1999, 2000] 
